In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.autonotebook import tqdm
from bracketology_rl import Bracket
from bracketgym import TournamentEnv
import warnings
import gym
import numpy as np
from tourney_tools import * 
warnings.filterwarnings('ignore')

/var/folders/jc/lq6pqgbn3dlfs9v5g5x4jc_80000gn/T/ipykernel_5332/2451804577.py:4: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [2]:

from stable_baselines3 import PPO,DDPG,A2C,TD3,SAC
from stable_baselines3.common.noise import NormalActionNoise
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.monitor import Monitor

from stable_baselines3.common.callbacks import EvalCallback
from stable_baselines3.common.env_util import make_vec_env


In [3]:
import tensorboard
## tensor board_log
tensorboard_log = "Logs/"

In [4]:
years = list(range(2003,2019))
year=years[1]

bracket = Bracket(year)

team_data=pd.read_csv('Process_data/pm_w_names.csv')

In [14]:
env=TournamentEnv(team_data=team_data)
ensemble=[PPO,PPO,PPO,PPO]
print(env.observation_space.shape)


(3616,)


In [15]:
action=(np.random.randn(env.action_space.shape[0]))
env.step(action)
print(env.observation_space.shape)

(3616,)


In [16]:
1e-4

0.0001

In [17]:
def make_model(model,env):
    model = model('MlpPolicy', env, verbose=0,
    learning_rate= 1e-4,
    gamma=  0.95,
    n_steps=6,
    tensorboard_log='Logs/')

    return model

In [18]:
models = [make_model(model,DummyVecEnv([lambda: env])) for model in ensemble]
models

In [19]:
best_params=[]
bar=tqdm(models,desc='Training')

for model in bar:
    model.set_env(env)
    try:
        model.learn(total_timesteps=100000)
    except KeyboardInterrupt:
        continue






Training: 100%|██████████| 4/4 [11:42:01<00:00, 10530.35s/it]


In [22]:
env=TournamentEnv(team_data=team_data,season=2022)




Reset! Sim 0 Bracket for year 2022:   0%|          | 0/120000 [00:07<?, ?it/s]


In [23]:

for i in range(1000):
    obs = env.reset(season=2022,weight_scores=False,track_wins=True)
    done = False
    while not done:
        actions = [model.predict(obs)[0] for model in models]
        # print(np.max(actions, axis=0))
        obs, reward, done, info = env.step(np.max(actions, axis=0))
env.run_final_bracket(season=2022)
env.bracket.score()
# ## test individually
# for model in models:
#     obs = env.reset(season=2022,weight_scores=False,track_wins=True)
#     done = False
#     while not done:
#         action = model.predict(obs)[0]
#         obs, reward, done, info = env.step(action)
    



Number of games correct: 34/63
Total Score: 56/192


In [24]:
env.run_final_bracket(season=2022)
print(env.bracket.score())

Number of games correct: 34/63
Total Score: 56/192
None
